In [ ]:
import re
import os
from IPython.display import Image
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import math
import matplotlib.ticker as plticker

class CustomRuntimeError(Exception):
    """Something failed in runtime"""
    def __init__(self,filename,message):
        self.filename = filename
        self.message = message

nvprof = "/usr/local/cuda-10.2/bin/nvprof"

def convolve(isize,fsize,static = 0):
    commands = [
        f"{nvprof} --normalized-time-unit ms ./filter_spec.out --isize={isize} --fsize={fsize} --test=0 2> filter_spec.txt",
        f"{nvprof} --normalized-time-unit ms ./filter.out --isize={isize} --fsize={fsize} --static={static} --test=0 2> filter.txt",
        
    ]
    for command in commands:
        if(os.system(command) == 0):
            continue
        raise CustomRuntimeError(command,"")

files = [
    "filter_spec.txt",
    "filter.txt",
]

def time():
    rePatt = re.compile('lambda_\w*|ConvolutionFilter\w*')
    flt = re.compile('[\d]+\.?[\d]*') 
    result = []
    for f in files:
        with open(f) as fr:
            val = 0
            for line in fr:
                if(rePatt.findall(line) != []):
                    val = val + (float(flt.findall(line)[1]))
            result.append(val)
    return result

filters1 = [15,31,63,127,255]
filters2 = [i for i in range(3,256,2)]

def test(iterations,static,filters):
    res = []
    for f in filters:
        res_ = []
        for i in range(iterations):
            convolve(16384,f,static)
            res_.append(time())
        res.append(res_)
    return res
    
print(test(5,0,filters2))

spec_vs_handtuned = [[[42.86486, 42.85684], [42.85841, 42.1094], [42.85996, 42.85328], [42.85872, 42.87062], [42.85438, 41.05494]], [[77.33528000000001, 77.31421999999999], [77.346, 77.33775], [77.34281, 74.08377], [77.33932, 71.87102], [77.34198, 69.9423]], [[145.19525, 121.1944], [146.29362, 146.25556999999998], [146.31086, 139.15553], [137.95949000000002, 123.98246], [139.75044, 129.9904]], [[360.98213, 303.68192999999997], [344.66498, 309.49604], [353.59361, 309.64422], [334.64797, 311.07076], [342.20041, 302.82372]], [[494.7473, 480.33979999999997], [506.3179, 485.116], [516.578, 480.30719999999997], [540.1828, 483.805], [499.4329, 487.1097]]]

def red(test_list):
    res = [0 for i in range(0,len(test_list[0]))];#12
    for i in test_list: #for each pair
        for j in range(0,len(res)):#7
            res[j]+=(i[j])
    return list(map(lambda x : x / len(test_list) ,res))

def build_set(test_list):
    res = [[] for i in range(0,len(test_list[0][0]))]
    for i in test_list: #for each filter size
        data = red(i) #average for specific run
        for j in range(0,len(test_list[0][0])):
            res[j].append(data[j])
    return res

def deviation(test_list,avg):
    dev = [[0 for i in range(len(test_list[0][0]))] for j in range(len(test_list))] #for each size
    for i in range(len(test_list)): #for each size
        for j in range(len(test_list[i])): #for each clock
            for e in range(len(test_list[i][j])):
                dev[i][e] = dev[i][e] + (((test_list[i][j][e] - avg[e][i])**2) / (len(test_list[i])-1)) #std dev
    res = [[] for i in range(len(avg))]
    for i in range(len(avg)): #for each algorithm
        for e in dev: #for each size
             res[i].append(math.sqrt(e[i]))
    return res

# print(build_set(spec_vs_handtuned))
# print(deviation(spec_vs_handtuned,build_set(spec_vs_handtuned)))

# yticks = [i for i in range(40,541,40)]
# plt.yticks(yticks)
# plt.xticks(filters1)
# plt.grid()
# avg_spec_hand = build_set(spec_vs_handtuned)
# spec_hand_div = deviation(spec_vs_handtuned,build_set(spec_vs_handtuned))

# mns_devplus = [[avg_spec_hand[j][i]+spec_hand_div[j][i] for i in range(len(avg_spec_hand[j]))] for j in range(len(avg_spec_hand))]
# mns_devminus = [[avg_spec_hand[j][i]-spec_hand_div[j][i] for i in range(len(avg_spec_hand[j]))] for j in range(len(avg_spec_hand))]


# plt.fill_between(filters1,mns_devplus[0],mns_devminus[0],interpolate=False,color='gray', alpha=0.3)
# plt.fill_between(filters1,mns_devplus[1],mns_devminus[1],interpolate=False,color='gray', alpha=0.3)

# plt.plot(filters1,avg_spec_hand[0],'o',label='separable_convolution_spec',color='violet',markersize=6)
# plt.plot(filters1,avg_spec_hand[1],'^',label='separable_convolution_cuda_handmade',color='red',markersize=6)


# plt.ylabel('time in ms')
# plt.xlabel('filter diameter')

# plt.legend()
# plt.show()
# plt.savefig('separable_convolution_spec_vs_hand.pdf')

